In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.preprocessing.image
# import sklearn.preprocessing
import os;
import datetime  
import cv2

import tensorflow as tf
# from data_loader import *
from abc import abstractmethod
from utils import *

import warnings
warnings.filterwarnings("ignore")

import random

Using TensorFlow backend.


In [3]:
import os
import numpy as np
def create_dir(dir_name, relative_path):
    """
    Create new directory if not exists
    --------------
    Parameters:
        dir_name (string)      - name of directory we want to create
        relative_path (string) - absolute path of directory we want to create
    Returns:
        path (string)          - full path of directory
    --------------
    """
    
    path = relative_path + dir_name
    if not os.path.exists(path):
        os.mkdir(path)
    return path

def normalize_data(data):
    """
    function to normalize data
    --------------
    Parameters:
        data (dataframe or matrix_like) - data we want to normalize, for example images
    Returns:
        dataframe or matrix_like (normalized data)
    --------------
    """
    # scale features using statistics that are robust to outliers
    #rs = sklearn.preprocessing.RobustScaler()
    #rs.fit(data)
    #data = rs.transform(data)
    #data = (data-data.mean())/(data.std()) # standardisation
    data = data / data.max() # convert from [0:255] to [0.:1.]
    #data = ((data / 255.)-0.5)*2. # convert from [0:255] to [-1.:+1.]
    return data

def one_hot_to_dense(labels_one_hot):
    """
    convert one-hot encodings into labels
    --------------
    Parameters:
        labels_one_hot (matrix_like) - all one hot vectors we want to encode
    Returns:
        int (encoded value)
    --------------
    """
    return np.argmax(labels_one_hot,1)

In [4]:
import os
import glob
from PIL import Image
import random
import numpy as np

class DataLoader:

    def __init__(self, train_images_dir, val_images_dir, test_images_dir, train_batch_size, val_batch_size, 
            test_batch_size, height_of_image, width_of_image, num_channels, num_classes):

        self.train_paths = glob.glob(os.path.join(train_images_dir, "**/*.png"), recursive=True)
        self.val_paths = glob.glob(os.path.join(val_images_dir, "**/*.png"), recursive=True)
        self.test_paths = glob.glob(os.path.join(test_images_dir, "**/*.png"), recursive=True)

#         random.shuffle(self.train_paths)
#         random.shuffle(self.val_paths)
#         random.shuffle(self.test_paths)

        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        self.test_batch_size = test_batch_size

        self.height_of_image = height_of_image
        self.width_of_image = width_of_image
        self.num_channels = num_channels
        self.num_classes = num_classes

    def load_image(self, path, is_flattened = False):
        im = np.asarray(Image.open(path))
        lbl = np.eye(self.num_classes)[int(path.rsplit('\\', 2)[-2])]

        if is_flattened:
            im = im.reshape(self.height_of_image * self.width_of_image)

        return im, lbl

    def batch_data_loader(self, batch_size, file_paths, index, is_flattened = False, randomization = False):
        ims = []
        lbls = []
        
        if index == 0 or randomization:
            random.shuffle(file_paths)
        
        while batch_size >= 1 and (len(file_paths) - index > 0):
            im, lbl = self.load_image(file_paths[index], is_flattened)
            ims.append(im)
            lbls.append(lbl)
            batch_size -= 1
            index += 1
        imgs = np.array(ims)
        imgs = imgs.reshape(-1, self.height_of_image, self.width_of_image, self.num_channels)

        return imgs, np.array(lbls)

    def train_data_loader(self, index, randomization = False):
        return self.batch_data_loader(self.train_batch_size, self.train_paths, index, randomization = randomization)

    def val_data_loader(self, index, randomization = False):
        return self.batch_data_loader(self.val_batch_size, self.val_paths, index, randomization = randomization)

    def test_data_loader(self, index, randomization = False):
        return self.batch_data_loader(self.test_batch_size, self.test_paths, index, randomization = randomization)
    
    def get_train_data_size(self):
        return len(self.train_paths)
    
    def get_val_data_size(self):
        return len(self.val_paths)
    
    def get_test_data_size(self):
        return len(self.test_paths)
    
    def all_train_data_loader(self, is_flattened = False):
        return self.batch_data_loader(self.get_train_data_size(), self.train_paths, 0)
    
    def all_val_data_loader(self, is_flattened = False):
        return self.batch_data_loader(self.get_val_data_size(), self.val_paths, 0)
    
    def all_test_data_loader(self, is_flattened = False):
        return self.batch_data_loader(self.get_test_data_size(), self.test_paths, 0)

In [5]:
# from .BaseNN import *

class DNN(BaseNN):

    def __init__(self, train_images_dir, val_images_dir, test_images_dir, num_epochs, train_batch_size,
                 val_batch_size, test_batch_size, height_of_image, width_of_image, num_channels, 
                 num_classes, learning_rate, base_dir, max_to_keep, model_name, keep_prob):

        super().__init__(train_images_dir, val_images_dir, test_images_dir, num_epochs, train_batch_size,
                 val_batch_size, test_batch_size, height_of_image, width_of_image, num_channels, 
                 num_classes, learning_rate, base_dir, max_to_keep, model_name, keep_prob)

        self.s_f_conv1 = 3; # filter size of first convolution layer (default = 3)
        self.n_f_conv1 = 36; # number of features of first convolution layer (default = 36)
        self.s_f_conv2 = 3; # filter size of second convolution layer (default = 3)
        self.n_f_conv2 = 36; # number of features of second convolution layer (default = 36)
        self.s_f_conv3 = 3; # filter size of third convolution layer (default = 3)
        self.n_f_conv3 = 36; # number of features of third convolution layer (default = 36)
        self.n_n_fc1 = 576; # number of neurons of first fully connected layer (default = 576)

    def weight_variable(self, shape, name = None):
        """
        Weight initialization
        -----------------
        Parameters:
            shape   (tuple)     - shape of weight variable
            name    (string)    - name of weight variable
        Returns:
            tf.Variable         - initialized weight variable
        -----------------
        """
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name = name)

    def bias_variable(self, shape, name = None):
        """
        Bias initialization
        -----------------
        Parameters:
            shape   (tuple)     - shape of bias variable
            name    (string)    - name of bias variable
        Returns:
            tf.Variable         - initialized bias variable
        -----------------
        """
        initial = tf.constant(0.1, shape=shape) #  positive bias
        return tf.Variable(initial, name = name)

    def conv2d(self, x, W, name = None):
        """
        2D convolution
        -----------------
        Parameters:
            x       (matrix-like)   - input data
            W       (tf.Variable)   - Weight matrix
            name    (string)        - name of the graph node
        Returns:
            tf.Conv                 - Convolution response
        -----------------
        """
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name = name)

    def max_pool_2x2(self, x, name = None):
        """
        2 x 2 max pooling
        -----------------
        Parameters:
            x       (matrix-like)   - matrix we want to apply max pooling
            name    (string)        - name of the graph node
        Returns:
            tf.max_pool             - max pooling response
        -----------------
        """
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name = name)

    def summary_variable(self, var, var_name):
        """
        Attach summaries to a tensor for TensorBoard visualization
        -----------------
        Parameters:
            var         - variable we want to attach
            var_name    - name of the variable
        Returns:
            None
        -----------------
        """
        with tf.name_scope(var_name):
            mean = tf.reduce_mean(var)
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('mean', mean)
            tf.summary.scalar('stddev', stddev)
            tf.summary.scalar('max', tf.reduce_max(var))
            tf.summary.scalar('min', tf.reduce_min(var))
            tf.summary.histogram('histogram', var)
        return None
    
    def load_tensors(self, graph):
        """
        load tensors from a saved graph
        -----------------
        Parameters:
            graph       (tf.graph_like) - graph we obtained from saved file
        Returns:
            None
        -----------------
        """

        # input tensors
        self.x_data_tf = graph.get_tensor_by_name("x_data_tf:0")
        self.y_data_tf = graph.get_tensor_by_name("y_data_tf:0")
        
        # weights and bias tensors
        self.W_conv1_tf = graph.get_tensor_by_name("W_conv1_tf:0")
        self.W_conv2_tf = graph.get_tensor_by_name("W_conv2_tf:0")
        self.W_conv3_tf = graph.get_tensor_by_name("W_conv3_tf:0")
        self.W_fc1_tf = graph.get_tensor_by_name("W_fc1_tf:0")
        self.W_fc2_tf = graph.get_tensor_by_name("W_fc2_tf:0")
        self.b_conv1_tf = graph.get_tensor_by_name("b_conv1_tf:0")
        self.b_conv2_tf = graph.get_tensor_by_name("b_conv2_tf:0")
        self.b_conv3_tf = graph.get_tensor_by_name("b_conv3_tf:0")
        self.b_fc1_tf = graph.get_tensor_by_name("b_fc1_tf:0")
        self.b_fc2_tf = graph.get_tensor_by_name("b_fc2_tf:0")
        
        # activation tensors
        self.h_conv1_tf = graph.get_tensor_by_name('h_conv1_tf:0')  
        self.h_pool1_tf = graph.get_tensor_by_name('h_pool1_tf:0')
        self.h_conv2_tf = graph.get_tensor_by_name('h_conv2_tf:0')
        self.h_pool2_tf = graph.get_tensor_by_name('h_pool2_tf:0')
        self.h_conv3_tf = graph.get_tensor_by_name('h_conv3_tf:0')
        self.h_pool3_tf = graph.get_tensor_by_name('h_pool3_tf:0')
        self.h_fc1_tf = graph.get_tensor_by_name('h_fc1_tf:0')
        self.z_pred_tf = graph.get_tensor_by_name('z_pred_tf:0')
        
        # training and prediction tensors
        self.learn_rate_tf = graph.get_tensor_by_name("learn_rate_tf:0")
        self.keep_prob_tf = graph.get_tensor_by_name("keep_prob_tf:0")
        self.cross_entropy_tf = graph.get_tensor_by_name('cross_entropy_tf:0')
        self.train_step_tf = graph.get_operation_by_name('train_step_tf')
        self.z_pred_tf = graph.get_tensor_by_name('z_pred_tf:0')
        self.y_pred_proba_tf = graph.get_tensor_by_name("y_pred_proba_tf:0")
        self.y_pred_correct_tf = graph.get_tensor_by_name('y_pred_correct_tf:0')
        self.accuracy_tf = graph.get_tensor_by_name('accuracy_tf:0')
        
        # tensor of stored losses and accuracies during training
        self.train_loss_tf = graph.get_tensor_by_name("train_loss_tf:0")
        self.train_acc_tf = graph.get_tensor_by_name("train_acc_tf:0")
        self.valid_loss_tf = graph.get_tensor_by_name("valid_loss_tf:0")
        self.valid_acc_tf = graph.get_tensor_by_name("valid_acc_tf:0")

        return None

    def attach_summary(self, sess):
        """
        Create summary tensors for tensorboard.
        -----------------
        Parameters:
            sess - the session for which we want to create summaries
        Returns:
            None
        -----------------
        """
        self.summary_variable(self.W_conv1_tf, 'W_conv1_tf')
        self.summary_variable(self.b_conv1_tf, 'b_conv1_tf')
        self.summary_variable(self.W_conv2_tf, 'W_conv2_tf')
        self.summary_variable(self.b_conv2_tf, 'b_conv2_tf')
        self.summary_variable(self.W_conv3_tf, 'W_conv3_tf')
        self.summary_variable(self.b_conv3_tf, 'b_conv3_tf')
        self.summary_variable(self.W_fc1_tf, 'W_fc1_tf')
        self.summary_variable(self.b_fc1_tf, 'b_fc1_tf')
        self.summary_variable(self.W_fc2_tf, 'W_fc2_tf')
        self.summary_variable(self.b_fc2_tf, 'b_fc2_tf')
        tf.summary.scalar('cross_entropy_tf', self.cross_entropy_tf)
        tf.summary.scalar('accuracy_tf', self.accuracy_tf)

        # merge all summaries for tensorboard
        self.merged = tf.summary.merge_all()

        # initialize summary writer 
        timestamp = datetime.datetime.now().strftime('%d-%m-%Y_%H-%M-%S')
        filepath = os.path.join(os.getcwd(), self.base_dir, self.model_name, 'logs', (self.model_name+'_'+timestamp))
        self.train_writer = tf.summary.FileWriter(os.path.join(filepath,'train'), sess.graph)
        self.valid_writer = tf.summary.FileWriter(os.path.join(filepath,'valid'), sess.graph)

        return None

    def network(self, X):
        """
        Construct network architecture
        -----------------
        Parameters:
            X       (tensor) - input data with shape of (batch size; height of image; width of image ; num channels)
        Returns:
            Last layer of the network (for continuation)
        -----------------
        """

        # 1.layer: convolution + max pooling
        self.W_conv1_tf = self.weight_variable([self.s_f_conv1, self.s_f_conv1, 1, self.n_f_conv1], name = 'W_conv1_tf') # (3,3,1,36)
        self.b_conv1_tf = self.bias_variable([self.n_f_conv1], name = 'b_conv1_tf') # (36)
        self.h_conv1_tf = tf.nn.relu(self.conv2d(X, self.W_conv1_tf) + self.b_conv1_tf, name = 'h_conv1_tf') # (.,28,28,36)
        self.h_pool1_tf = self.max_pool_2x2(self.h_conv1_tf, name = 'h_pool1_tf') # (.,14,14,36)

        # 2.layer: convolution + max pooling
        self.W_conv2_tf = self.weight_variable([self.s_f_conv2, self.s_f_conv2, self.n_f_conv1, self.n_f_conv2], name = 'W_conv2_tf')
        self.b_conv2_tf = self.bias_variable([self.n_f_conv2], name = 'b_conv2_tf')
        self.h_conv2_tf = tf.nn.relu(self.conv2d(self.h_pool1_tf, self.W_conv2_tf) + self.b_conv2_tf, name ='h_conv2_tf') #(.,14,14,36)
        self.h_pool2_tf = self.max_pool_2x2(self.h_conv2_tf, name = 'h_pool2_tf') #(.,7,7,36)

        # 3.layer: convolution + max pooling
        self.W_conv3_tf = self.weight_variable([self.s_f_conv3, self.s_f_conv3, self.n_f_conv2, self.n_f_conv3], name = 'W_conv3_tf')
        self.b_conv3_tf = self.bias_variable([self.n_f_conv3], name = 'b_conv3_tf')
        self.h_conv3_tf = tf.nn.relu(self.conv2d(self.h_pool2_tf, self.W_conv3_tf) + self.b_conv3_tf, name = 'h_conv3_tf') #(.,7,7,36)
        self.h_pool3_tf = self.max_pool_2x2(self.h_conv3_tf, name = 'h_pool3_tf') # (.,4,4,36)

        # 4.layer: fully connected
        self.W_fc1_tf = self.weight_variable([4*4*self.n_f_conv3,self.n_n_fc1], name = 'W_fc1_tf') # (4*4*36, 576)
        self.b_fc1_tf = self.bias_variable([self.n_n_fc1], name = 'b_fc1_tf') # (576)
        self.h_pool3_flat_tf = tf.reshape(self.h_pool3_tf, [-1,4*4*self.n_f_conv3], name = 'h_pool3_flat_tf') # (.,576)
        self.h_fc1_tf = tf.nn.relu(tf.matmul(self.h_pool3_flat_tf, self.W_fc1_tf) + self.b_fc1_tf, name = 'h_fc1_tf') # (.,576)
      
        # add dropout
        self.keep_prob_tf = tf.placeholder(dtype=tf.float32, name = 'keep_prob_tf')
        self.h_fc1_drop_tf = tf.nn.dropout(self.h_fc1_tf, self.keep_prob_tf, name = 'h_fc1_drop_tf')

        # 5.layer: fully connected
        self.W_fc2_tf = self.weight_variable([self.n_n_fc1, 10], name = 'W_fc2_tf')
        self.b_fc2_tf = self.bias_variable([10], name = 'b_fc2_tf')
        
        self.z_pred_tf = tf.add(tf.matmul(self.h_fc1_drop_tf, self.W_fc2_tf), self.b_fc2_tf, name = 'z_pred_tf')# => (.,10)

        return self.z_pred_tf

    def metrics(self, Y, Y_pred):
        """
        Some metric, here I use simple accuracy
        -----------------
        Parameters:
            Y       (array_like) - actual labels
            Y_pred  (array_like) - predicted labels
        Returns:
            Float (Accuracy)
        -----------------
        """

        Y = Y.reshape(-1,)
        Y_pred = Y_pred.reshape(-1,)
        return np.mean(Y == Y_pred)

In [6]:
import tensorflow as tf
# from data_loader import *
from utils import *
from abc import abstractmethod
import numpy as np
import pandas as pd
import os;
import datetime  
import cv2

class BaseNN:
    def __init__(self, train_images_dir, val_images_dir, test_images_dir, num_epochs, train_batch_size,
                 val_batch_size, test_batch_size, height_of_image, width_of_image, num_channels, 
                 num_classes, learning_rate, base_dir, max_to_keep, model_name, keep_prob):

        self.data_loader = DataLoader(train_images_dir, val_images_dir, test_images_dir, train_batch_size, 
                val_batch_size, test_batch_size, height_of_image, width_of_image, num_channels, num_classes)

        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.base_dir = base_dir
        self.max_to_keep = max_to_keep
        self.model_name = model_name
        self.keep_prob = keep_prob

        self.n_log_step = 0 # counting current number of mini batches trained on
    
    def create_network(self):
        """
        Create base components of the network.
        Main structure of network will be described in network function.
        -----------------
        Parameters:
            None
        Returns:
            None
        -----------------
        """
        tf.reset_default_graph()

        # variables for input and output 
        self.x_data_tf = tf.placeholder(dtype=tf.float32, shape=[None, self.data_loader.height_of_image, self.data_loader.width_of_image, self.data_loader.num_channels], name='x_data_tf')
        self.y_data_tf = tf.placeholder(dtype=tf.float32, shape=[None, self.data_loader.num_classes], name='y_data_tf')

        self.z_pred_tf = self.network(self.x_data_tf)

        # cost function
        self.cross_entropy_tf = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=self.y_data_tf, logits=self.z_pred_tf), name = 'cross_entropy_tf')

        # optimisation function
        self.learn_rate_tf = tf.placeholder(dtype=tf.float32, name="learn_rate_tf")
        self.train_step_tf = tf.train.AdamOptimizer(self.learn_rate_tf).minimize(
            self.cross_entropy_tf, name = 'train_step_tf')

        # predicted probabilities in one-hot encoding
        self.y_pred_proba_tf = tf.nn.softmax(self.z_pred_tf, name='y_pred_proba_tf') 
        
        # tensor of correct predictions
        self.y_pred_correct_tf = tf.equal(tf.argmax(self.y_pred_proba_tf, 1),
                                          tf.argmax(self.y_data_tf, 1),
                                          name = 'y_pred_correct_tf')  
        
        # accuracy 
        self.accuracy_tf = tf.reduce_mean(tf.cast(self.y_pred_correct_tf, dtype=tf.float32),
                                         name = 'accuracy_tf')

        # tensors to save intermediate accuracies and losses during training
        self.train_loss_tf = tf.Variable(np.array([]), dtype=tf.float32, 
                                         name='train_loss_tf', validate_shape = False)
        self.valid_loss_tf = tf.Variable(np.array([]), dtype=tf.float32,
                                         name='valid_loss_tf', validate_shape = False)
        self.train_acc_tf = tf.Variable(np.array([]), dtype=tf.float32, 
                                        name='train_acc_tf', validate_shape = False)
        self.valid_acc_tf = tf.Variable(np.array([]), dtype=tf.float32, 
                                        name='valid_acc_tf', validate_shape = False)

        return None

    def close_writers(self):
        """
        Close train and validation summary writer.
        -----------------
        Parameters:
            sess - the session we want to save
        Returns:
            None
        -----------------
        """
        self.train_writer.close()
        self.valid_writer.close()

        return None

    def save_model(self, sess):
        """
        Save tensors/summaries
        -----------------
        Parameters:
            sess - the session we want to save
        Returns:
            None
        -----------------
        """
        filepath = os.path.join(os.getcwd(), self.base_dir, self.model_name, 'checkpoints', self.model_name)
        self.saver_tf.save(sess, filepath)
        
        return None

    def train_model_helper(self, sess, n_epoch = 1):
        """
        Helper function to train the model.
        -----------------
        Parameters:
            sess - current session
            n_epoch (int)         - number of epochs
        Returns:
            None
        -----------------
        """
        
        train_loss, train_acc, valid_loss, valid_acc = [],[],[],[]
        
        # start timer
        start = datetime.datetime.now()
        print(datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S'),': start training')
        print('learnrate = ', self.learning_rate,', n_epoch = ', n_epoch,
              ', mb_size = ', self.data_loader.train_batch_size)
        
        # looping over epochs
        for e in range(1, n_epoch+1):
            index = 0
            # looping over mini batches
            for i in range(1, int(np.ceil(self.data_loader.get_train_data_size() / self.data_loader.train_batch_size))+1):
                x_batch, y_batch = self.data_loader.train_data_loader(index)
                x_valid, y_valid = self.data_loader.val_data_loader(0, randomization = True)
                
                self.sess.run(self.train_step_tf,feed_dict={self.x_data_tf: x_batch,
                                                            self.y_data_tf: y_batch,
                                                            self.keep_prob_tf: self.keep_prob,
                                                            self.learn_rate_tf: self.learning_rate})

                feed_dict_valid = {self.x_data_tf: x_valid,
                                   self.y_data_tf: y_valid,
                                   self.keep_prob_tf: 1.0}

                feed_dict_train = {self.x_data_tf: x_batch,
                                    self.y_data_tf: y_batch,
                                    self.keep_prob_tf: 1.0}
                
                # store losses and accuracies
                if i%self.validation_step == 0:
                    valid_loss.append(sess.run(self.cross_entropy_tf,
                                               feed_dict = feed_dict_valid))
                    valid_acc.append(self.accuracy_tf.eval(session = sess, 
                                                           feed_dict = feed_dict_valid))
                    print('%.0f epoch, %.0f iteration: val loss = %.4f, val acc = %.4f'%(
                        e, i, valid_loss[-1],valid_acc[-1]))

                # summary for tensorboard
                if i%self.summary_step == 0:
                    self.n_log_step += 1 # for logging the results
                    train_summary = sess.run(self.merged, feed_dict={self.x_data_tf: x_batch, 
                                                                    self.y_data_tf: y_batch, 
                                                                    self.keep_prob_tf: 1.0})
                    valid_summary = sess.run(self.merged, feed_dict = feed_dict_valid)
                    self.train_writer.add_summary(train_summary, self.n_log_step)
                    self.valid_writer.add_summary(valid_summary, self.n_log_step)

                if i%self.display_step == 0:
                    train_loss.append(sess.run(self.cross_entropy_tf,
                                               feed_dict = feed_dict_train))
                    train_acc.append(self.accuracy_tf.eval(session = sess, 
                                                           feed_dict = feed_dict_train))
                    print('%.0f epoch, %.0f iteration: train loss = %.4f, train acc = %.4f'%(
                        e, i,  train_loss[-1],train_acc[-1]))

                # save current model to disk
                if i%self.checkpoint_step == 0:
                    self.save_model(sess)
                
                index += self.data_loader.train_batch_size
                
        # concatenate losses and accuracies and assign to tensor variables
        tl_c = np.concatenate([self.train_loss_tf.eval(session=sess), train_loss], axis = 0)
        vl_c = np.concatenate([self.valid_loss_tf.eval(session=sess), valid_loss], axis = 0)
        ta_c = np.concatenate([self.train_acc_tf.eval(session=sess), train_acc], axis = 0)
        va_c = np.concatenate([self.valid_acc_tf.eval(session=sess), valid_acc], axis = 0)
   
        sess.run(tf.assign(self.train_loss_tf, tl_c, validate_shape = False))
        sess.run(tf.assign(self.valid_loss_tf, vl_c , validate_shape = False))
        sess.run(tf.assign(self.train_acc_tf, ta_c , validate_shape = False))
        sess.run(tf.assign(self.valid_acc_tf, va_c , validate_shape = False))
        
        print('running time for training: ', datetime.datetime.now() - start)
        return None
    
    def train_model(self, display_step, validation_step, checkpoint_step, summary_step):
        """
        Main function for model training.
        -----------------
        Parameters:
            display_step       (int)   - Number of steps we cycle through before displaying detailed progress
            validation_step    (int)   - Number of steps we cycle through before validating the model
            checkpoint_step    (int)   - Number of steps we cycle through before saving checkpoint
            summary_step       (int)   - Number of steps we cycle through before saving summary
        Returns:
            None
        -----------------
        """
        self.display_step = display_step
        self.validation_step = validation_step
        self.checkpoint_step = checkpoint_step
        self.summary_step = summary_step
        
        # self.x_valid, self.y_valid = self.data_loader.all_val_data_loader()

        self.saver_tf = tf.train.Saver(max_to_keep = self.max_to_keep)

        # attach summaries
        self.attach_summary(self.sess)

        # training on original data
        self.train_model_helper(self.sess, n_epoch = self.num_epochs)

        # save final model
        self.save_model(self.sess)

        self.close_writers()

    def get_accuracy(self, sess):
        """
        Get accuracies of training and validation sets.
        -----------------
        Parameters:
            sess - session
        Returns:
            tuple (tuple of lists) train and validation accuracies
        -----------------
        """
        train_acc = self.train_acc_tf.eval(session = sess)
        valid_acc = self.valid_acc_tf.eval(session = sess)
        return train_acc, valid_acc

    def get_loss(self, sess):
        """
        Get losses of training and validation sets.
        -----------------
        Parameters:
            sess - session
        Returns:
            tuple (tuple of lists) train and validation losses
        -----------------
        """
        train_loss = self.train_loss_tf.eval(session = sess)
        valid_loss = self.valid_loss_tf.eval(session = sess)
        return train_loss, valid_loss 

    def forward(self, sess, x_data):
        """
        Forward prediction of current graph.
        Will be used in test_model method.
        -----------------
        Parameters:
            sess                 - actual session
            x_data (matrix_like) - data for which we want to calculate predicted probabilities
        Returns:
            vector_like - predicted probabilities for input data
        -----------------
        """
        y_pred_proba = self.y_pred_proba_tf.eval(session = sess, 
                                                 feed_dict = {self.x_data_tf: x_data,
                                                              self.keep_prob_tf: 1.0})
        return y_pred_proba
    
    def load_session_from_file(self, filename):
        """
        Load session from file, restore graph, and load tensors.
        -----------------
        Parameters:
            filename (string) - the name of the model (name of file we saved in disk)
        Returns:
            session
        -----------------
        """
        tf.reset_default_graph()

        filepath = os.path.join(os.getcwd(), filename + '.meta')
        # filepath = os.path.join(os.getcwd(), self.base_dir, filename + '.meta')
        # filepath = os.path.join(os.getcwd(), self.base_dir, self.model_name, filename + '.meta')
        # filepath = os.path.join(os.getcwd(), self.base_dir, self.model_name, 'checkpoints', filename + '.meta')
        print(filepath)
        saver = tf.train.import_meta_graph(filepath)
        sess = tf.Session()
        saver.restore(sess, self.model_name)
        graph = tf.get_default_graph()
        
        self.load_tensors(graph)
        
        return sess

    def test_model(self):
        """
        load model and test on test data.
        -----------------
        Parameters:
            None
        Returns:
            metric, defined in dnn class (for example accuracy)
        -----------------
        """
        x_test, y_test = self.data_loader.all_test_data_loader()
                
        sess = self.load_session_from_file(self.model_name)
        
        y_test_pred = {self.model_name : []}
        y_test_pred_labels = {}
        
        accuracies = []
        
        # looping over mini batches
        index = 0
        for i in range(1, int(np.ceil(self.data_loader.get_test_data_size() / self.data_loader.test_batch_size))+1):
            x_test, y_test = self.data_loader.test_data_loader(index)
            y_test_pred[self.model_name].append(self.forward(sess, x_test))
#             y_test_pred[self.model_name] = self.forward(sess, x_test)
            
            y_test_pred_labels[self.model_name] = one_hot_to_dense(y_test_pred[self.model_name])
            y_test = one_hot_to_dense(y_test)
            
            accuracies.append(self.metrics(y_test, y_test_pred_labels[self.model_name]))
        
#         for i in y_test_pred.values():
#             y_test_pred_ = np.concatenate( i, axis=0 )
        
        sess.close()
        
        return accuracies
        
#         return y_test_pred_
    
#         y_test_pred_labels[self.model_name] = one_hot_to_dense(y_test_pred[self.model_name])
#         y_test = one_hot_to_dense(y_test)
        
#         print('Test Accuracy: ', self.metrics(y_test, y_test_pred_labels[self.model_name]))
#         return self.metrics(y_test, y_test_pred_labels[self.model_name])

    def initialize_network(self):
        """
        Initialize network from last checkpoint if exists, otherwise initialize with random values.
        -----------------
        Parameters:
            None
        Returns:
            metric, defined in dnn class (for example accuracy)
        -----------------
        """
        self.sess = tf.InteractiveSession()
        filepath = os.path.join(os.getcwd(), self.base_dir, self.model_name, 'checkpoints', self.model_name + '.meta')
        if ~os.path.isdir(filepath):
            self.sess.run(tf.global_variables_initializer())
        else:
            self.sess = self.load_session_from_file(self.model_name)
        return None
    
    @abstractmethod
    def network(self, X):
        raise NotImplementedError('subclasses must override network()!')

    @abstractmethod
    def metrics(self, Y, y_pred):
        raise NotImplementedError('subclasses must override metrics()!')

In [7]:
nn_graph = DNN(
    train_images_dir='C:/_Files/MyProjects/ASDS_3/ASDS_DL/Homeworks/3_mnist/_inputs_image/all/train/image/',
    val_images_dir='C:/_Files/MyProjects/ASDS_3/ASDS_DL/Homeworks/3_mnist/_inputs_image/all/validation/image/',
    test_images_dir='C:/_Files/MyProjects/ASDS_3/ASDS_DL/Homeworks/3_mnist/_inputs_image/all/test/image/',
    num_epochs=2,
    train_batch_size=100,
    val_batch_size=100,
    test_batch_size=1000,
    height_of_image=28,
    width_of_image=28,
    num_channels=1,
    num_classes=10,
    learning_rate = 0.001,
    base_dir='results',
    max_to_keep=5,
    model_name='nn_1',
    keep_prob=0.33
)

In [8]:
nn_graph.create_network()

W1106 15:23:17.058592  6436 deprecation.py:506] From <ipython-input-5-ece43f5a3479>:223: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1106 15:23:17.076544  6436 deprecation.py:323] From <ipython-input-1-2032920d05de>:49: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
nn_graph.initialize_network()

In [10]:
# display_step, validation_step, checkpoint_step, summary_step
# nn_graph.train_model(100, 100, 100, 100)

In [11]:
ddxk = nn_graph.test_model()

W1106 15:23:22.018013  6436 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


C:\_Files\MyProjects\ASDS_3\ASDS_DL\Homeworks\3_mnist\Junk Codes\nn_1.meta


In [12]:
ddxk

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
a = np.array([[1,2,],[4, 5, 6]])
b = np.array([[1,2,3],[4, 5, 6]])

In [ ]:
np.concatenate((a, b))

In [ ]:
a + b

In [ ]:
from collections import defaultdict

# for lists
d = defaultdict(list)
d['C1'].append([1, 2])

In [ ]:
d['C1'].append([1, 2])

In [ ]:
a = np.array([[1,2,3]])
d = {'index': []}
d['index'].extend(list(a))

In [ ]:
d['index'].extend(list(a))

In [ ]:
a = np.array([[1,2,3], [1,2,5]])

In [ ]:
list(a)